In [33]:
from pymongo import MongoClient
import pymongo
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pickle
from tmqr.settings import *

In [7]:
instrument = 'TST'
market = 'T'
future = 'F-TST-H11-110322'
opt_expiration = '110322'

In [20]:
options = ['C', 'P']
N = 60

In [10]:
strikes = list(np.arange(50.0, 100.0, 1.0))

In [18]:
options_tickers_list = []
for opt_type in options:
    for s in strikes:
        #US.C.F-CL-H11-110322.110121@89.0
        options_tickers_list.append("{0}.{1}.{2}.{3}@{4}".format(market, opt_type, future,  opt_expiration, s))        

In [84]:
options_tickers_list[0]

'T.C.F-TST-H11-110322.110322@50.0'

In [97]:
dates = [datetime(2011, 1, 2) + timedelta(days=x) for x in range(N)]

client = MongoClient(MONGO_CONNSTR)
db = client[MONGO_DB]

for i, opt in enumerate(options_tickers_list):
    iv_arr = np.random.random(size=N)
    price_arr = np.random.random(size=N)
    to_exp = np.arange(N)
    
    df = pd.DataFrame({'iv': iv_arr, 'px': price_arr, 'toexp': to_exp}, index=dates)
    #
    # Writing bundled quotes
    # 
    db['quotes_options_eod_bundled'].create_index([('tckr', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].create_index([('idx', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].replace_one({'tckr': opt},
                                                 {'tckr': opt, 
                                                  'data': pickle.dumps(df),
                                                  'idx': i,
                                                 },
                                                 upsert=True
    )
    #
    # Writing single records
    # 
    db['quotes_options_eod_single'].create_index([('tckr', pymongo.ASCENDING), ('dt', pymongo.ASCENDING)])
    db['quotes_options_eod_single'].create_index([('idx', pymongo.ASCENDING)])
    for i, dt in enumerate(dates):
        """print({
            'tckr': opt,
            'dt': dt,
            'iv': iv_arr[i],
            'px': price_arr[i],
            'to_exp': to_exp[i]
        })
        """
        db['quotes_options_eod_single'].replace_one({'tckr': opt, 'dt': dt},
            {
                'tckr': opt,
                'dt': dt,
                'iv': iv_arr[i],
                'px': price_arr[i],
                'idx': i,
                #'to_exp': to_exp[i]
            }, upsert=True)
    
    
    
    

In [119]:
class ChainSingle:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        
    def compose(self, date):
        result = db['quotes_options_eod_single'].find({'tckr': {'$in': self.tickers}, 'dt': date})
        #result = db['quotes_options_eod_single'].find({'idx': {'$in': list(range(len(self.tickers)))}, 'dt': date})
        return pd.DataFrame(list(result))
    
    def get_one(self, tckr, date):
        result = db['quotes_options_eod_single'].find({'tckr': tckr, 'dt': date})
        return pd.DataFrame(list(result))
        
class ChainBundled:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        self.panel = None
        self.one_df = None
        
    def compose(self, date):
        if self.panel is None:
            result = db['quotes_options_eod_bundled'].find({'tckr': {'$in': self.tickers}})  
            #result = db['quotes_options_eod_bundled'].find({'idx': {'$in': list(range(len(self.tickers)))}})  
            
            res_dict = {}
            for res in result:
                res_dict[res['tckr']] = pickle.loads(res['data'])

            self.panel = pd.Panel(res_dict)
        return self.panel.major_xs(date)
    
    def get_one(self, tckr, date):
        if self.one_df is None:
            result = db['quotes_options_eod_bundled'].find_one({'tckr': tckr})        
        
            self.one_df = pickle.loads(result['data'])
        return self.one_df.loc[date]
        
        
        

# Chaing composition benchmark

In [107]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.compose(datetime(2011, 1, 2))

100 loops, best of 3: 5.6 ms per loop


In [108]:
%%timeit
chain2 = ChainBundled(options_tickers_list)
chain2.compose(datetime(2011, 1, 2))

10 loops, best of 3: 80.1 ms per loop


In [109]:
%timeit chain1.compose(datetime(2011, 1, 2))

100 loops, best of 3: 4.61 ms per loop


In [115]:
%timeit chain2.compose(datetime(2011, 1, 3))

10000 loops, best of 3: 94.6 µs per loop


In [116]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.compose(datetime(2011, 1, 2) + timedelta(days=i))

10 loops, best of 3: 99.9 ms per loop


In [117]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.compose(datetime(2011, 1, 2) + timedelta(days=i))

10 loops, best of 3: 80.4 ms per loop


# Single element fetching benchmark

In [112]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

100 loops, best of 3: 3.14 ms per loop


In [113]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

100 loops, best of 3: 2.34 ms per loop


In [92]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

10 loops, best of 3: 21.5 ms per loop


In [93]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

100 loops, best of 3: 6.29 ms per loop


In [95]:
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

iv       0.766215
px       0.977808
toexp    0.000000
Name: 2011-01-02 00:00:00, dtype: float64